In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
import os
import json

# Define Google Drive API authentication
SERVICE_ACCOUNT_FILE = '../credentials.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

flow = InstalledAppFlow.from_client_secrets_file(SERVICE_ACCOUNT_FILE, SCOPES)
creds = flow.run_local_server(port=8502)
service = build('drive', 'v3', credentials=creds)

# Google Drive folder IDs
KEYFRAME_FOLDER_ID = '1bqJG0CRIIuVIib3pBcA2k8iiRyWlwmq9'

def list_drive_files(folder_id):
    """List all files and folders within a specified Google Drive folder."""
    query = f"'{folder_id}' in parents and trashed = false"
    files = []
    try:
        results = service.files().list(q=query, fields="files(id, name, mimeType)").execute()
        files = results.get('files', [])
    except HttpError as error:
        print(f"An error occurred: {error}")
    return files

def get_image_paths(folder_id, video_ID):
    """Retrieve image information from a Google Drive folder."""
    files = list_drive_files(folder_id)
    image_video_dict = {}
    idx = 0
    for file in sorted(files, key=lambda x: x['name']):
        if file['mimeType'] == 'image/jpeg':
            file_id = file['id']
            img_url = f"https://drive.google.com/uc?id={file_id}"  # Public link format
            frame_timestamp = file['name'].replace('.jpg', '')
            image_video_dict[idx] = {'frame_path': img_url, 'video_ID': video_ID, 'timestamp': frame_timestamp}
            idx += 1
    return image_video_dict

def save_json(data, output_folder, filename):
    """Save JSON data to a file."""
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    file_path = os.path.join(output_folder, filename)
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)

def process_keyframe_folders(keyframe_folder_id, output_folder):
    """Process all keyframe folders online using Google Drive API."""
    video_folders = list_drive_files(keyframe_folder_id)
    for video_folder in video_folders:
        if video_folder['mimeType'] == 'application/vnd.google-apps.folder':  # Ensure it's a folder
            video_ID = video_folder['name']  # e.g., "Video0"
            video_folder_id = video_folder['id']

            subfolders = list_drive_files(video_folder_id)
            for subfolder in subfolders:
                if '_reduced' in subfolder['name']:
                    continue
                
                subfolder_id = subfolder['id']
                image_info_dict = get_image_paths(subfolder_id, video_ID)
                
                filename = f"{subfolder['name']}.json"
                save_json(image_info_dict, output_folder, filename)

output_folder = '../keyframe_information/annotation'  # Local path to save JSON outputs
process_keyframe_folders(KEYFRAME_FOLDER_ID, output_folder)